# ДЗ 3. TensorFlow

1. Постройте нейронную сеть (берем несложную полносвязную сеть, меняем число слоев, число нейронов, типы активации, тип оптимизатора) на датасете from sklearn.datasets import load_boston.  
2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу. Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import seaborn as sns

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD

from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.losses import MeanAbsoluteError

%matplotlib inline

**Обзор данных**

In [2]:
# Загрузка данных
df_boston = load_boston()
boston = pd.DataFrame(df_boston.data, columns=df_boston.feature_names)
boston['TARGET'] = df_boston.target
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,TARGET
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  TARGET   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [4]:
boston.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
CRIM,506.0,3.613524,8.601545,0.00632,0.082045,0.25651,3.677083,88.9762
ZN,506.0,11.363636,23.322453,0.00000,0.000000,0.00000,12.500000,100.0000
INDUS,506.0,11.136779,6.860353,0.46000,5.190000,9.69000,18.100000,27.7400
CHAS,506.0,0.069170,0.253994,0.00000,0.000000,0.00000,0.000000,1.0000
NOX,506.0,0.554695,0.115878,0.38500,0.449000,0.53800,0.624000,0.8710
RM,506.0,6.284634,0.702617,3.56100,5.885500,6.20850,6.623500,8.7800
AGE,506.0,68.574901,28.148861,2.90000,45.025000,77.50000,94.075000,100.0000
DIS,506.0,3.795043,2.105710,1.12960,2.100175,3.20745,5.188425,12.1265
RAD,506.0,9.549407,8.707259,1.00000,4.000000,5.00000,24.000000,24.0000
TAX,506.0,408.237154,168.537116,187.00000,279.000000,330.00000,666.000000,711.0000


In [5]:
X = boston.drop(['TARGET'], axis=1)
y = boston['TARGET']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train.shape, X_test.shape

((379, 13), (127, 13))

**Построение нейросети**

In [7]:
result = pd.DataFrame(columns=['Слои', 'Нейроны', 'Оптимизатор', 'Train', 'Test'])

for i in range(3):

    if i==0: neurons = [64, 128, 256]
    if i==1: neurons = [[64, 16], [128, 32], [256, 64]]
    if i==2: neurons = [[64, 32, 32, 16], [128, 64, 32, 16], [256, 128, 64, 32]]

    optimizers = ['SGD', 'RMSprop', 'Adam']

    for neuron in neurons:
        for optim in optimizers:

            inputs = Input(shape=(13))
            
            if i==0:
                x = Dense(neuron, activation='relu')(inputs)
                outputs = Dense(1)(x)
                nlayer = 2
            
            if i==1:
                x = Dense(neuron[0], activation='relu')(inputs)
                x_1 = Dense(neuron[1], activation='relu')(x)
                outputs = Dense(1)(x_1)
                nlayer = 3
                
            if i==2:
                x = Dense(neuron[0], activation='relu')(inputs)
                x_1 = Dense(neuron[1], activation='relu')(x)
                x_2 = Dense(neuron[2], activation='relu')(x_1)
                x_3 = Dense(neuron[3], activation='relu')(x_2)
                outputs = Dense(1)(x_3)
                nlayer = 5

            model = Model(inputs, outputs)
            model.compile(optimizer=optim, loss='mae', metrics=['mae'])

            model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

            loss_train, metric_train = model.evaluate(X_train, y_train, batch_size=10)
            loss_test, metric_test = model.evaluate(X_test, y_test, batch_size=10)

            new_row = {'Слои': nlayer, 'Нейроны': neuron, 'Оптимизатор': optim, \
                       'Train': round(metric_train, 3), 'Test': round(metric_test,3)}

            result = result.append(new_row, ignore_index=True)
            
    print('\n' + '=' * 100 + '\n')

13/13 [==============================] - 0s 333us/step - loss: 6.2370 - mae: 6.2370


13/13 [==============================] - 0s 389us/step - loss: 3.4975 - mae: 3.4975


13/13 [==============================] - 0s 416us/step - loss: 2.8410 - mae: 2.8410




In [8]:
result

,Слои,Нейроны,Оптимизатор,Train,Test
0,2,64,SGD,6.484,6.718
1,2,64,RMSprop,4.600,5.068
2,2,64,Adam,3.073,3.767
3,2,128,SGD,6.484,6.718
4,2,128,RMSprop,4.285,4.942
5,2,128,Adam,3.201,3.974
6,2,256,SGD,6.485,6.719
7,2,256,RMSprop,4.252,4.900
8,2,256,Adam,5.946,6.237
9,3,"[64, 16]",SGD,6.484,6.718


In [9]:
result['Нейроны'] = result['Нейроны'].astype(np.str) 
stats_neurons = result[['Нейроны', 'Train', 'Test']].groupby(['Нейроны'],as_index=False).mean().sort_values('Test')
stats_neurons

,Нейроны,Train,Test
5,"[256, 128, 64, 32]",2.712000,3.030500
8,"[64, 32, 32, 16]",2.939000,3.425000
4,"[128, 64, 32, 16]",4.055667,4.403333
6,"[256, 64]",4.104000,4.428667
7,"[64, 16]",4.322333,4.600667
2,64,4.719000,5.184333
0,128,4.656667,5.211333
3,"[128, 32]",5.139667,5.331667
1,256,5.561000,5.952000


In [10]:
stats_optimizers = result[['Оптимизатор', 'Train', 'Test']].groupby(['Оптимизатор'],as_index=False).mean().\
                   sort_values('Test')
stats_optimizers

,Оптимизатор,Train,Test
0,Adam,3.579333,3.915889
1,RMSprop,3.480889,3.991333
2,SGD,6.490714,6.726000


**Выводы**  

- Увеличение числа нейронов, улучшает качество рузультатов в нашей нейросети  
- Как обычно, лучше всего себя показал оптимизатор - Adam? он показал максимальную точность  